In [15]:
import pandas as pd
import json
import csv
import pandas as pd
from datetime import datetime


with open('/home/chuhan/ICWSM2023/metadata/Information2.json', encoding='utf-8') as f:
    students = json.load(f)

with open('/home/chuhan/ICWSM2023/metadata/Tweets_2.json', encoding='utf-8') as f:
    tweets = json.load(f)
    
training_tweets = []
label = []
for i in students:
    id = int(i["id"])
    print(id)
    self_identity_status = i["self_identity_status"]
    label.append(self_identity_status)
    training_tweet=""
    for j in tweets:
        if j["user_id"]==id:
            create = datetime.strptime(j["created_at"][:10], '%Y-%m-%d')
            if create>=(datetime.strptime("2020-12-01", '%Y-%m-%d')) and create<= (datetime.strptime("2020-12-31", '%Y-%m-%d')):
                #training_tweet = training_tweet + " " + j["pure_content_with_punc"] + " " + j["pure_origin_content_with_punc"]
                training_tweet = training_tweet + " " + j["pure_content_with_punc"]
                
    training_tweets.append(training_tweet)
    
dataframe = pd.DataFrame({'tweets':training_tweets,'self-identity':label})
    
dataframe.to_csv("/home/chuhan/ICWSM2023/tweets_only/dataforBert2.csv",index=False,sep=',')   
    

1066254584
1221035013
1774380377
1819995113
1826650570
1908753862
2019360341
2134254597
2144559370
2230716672
2251229954
2274381983
2284799541
2308867224
2350217967
2362292364
2507193140
2548688754
2563352957
2564163174
2576418994
2577442111
2583568215
2651685791
2657767595
2667910241
2725019565
2725882157
2732015163
2774232377
2774858232
2782907607
2787353101
2805132967
2815148025
2824120834
2859325984
2881860924
2885774884
2927947415
2972761071
3026226512
3059644332
3190725355
3193916592
3198145737
3225804421
3241065893
3273723102
3399558770
3490956272
3538034171
3579554510
3613309420
3634774787
3637375571
3682227410
3687425037
3735645817
3737701995
3738648602
3744061992
3830003729
3942660991
3979370545
3984091869
3990021151
3990051720
3992483005
5019025338
5026346571
5034056660
5036698178
5075090915
5092494244
5108783046
5124566623
5124988618
5127113163
5129132663
5142775047
5147840124
5161695030
5181986595
5212401864
5212457961
5224366105
5228371170
5232665012
5239880332
5242712424

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd
import csv

data = pd.read_csv("/home/chuhan/ICWSM2023/tweets+retweets/dataforBert2.csv", header=0)
def train_valid_test_split(x_data, y_data,
                           validation_size=0.2, test_size=0.2, shuffle=True):
    x_, x_test, y_, y_test = train_test_split(x_data, y_data, test_size=test_size, random_state=42, shuffle=shuffle)
    valid_size = validation_size / (1.0 - test_size)
    x_train, x_valid, y_train, y_valid = train_test_split(x_, y_, test_size=valid_size, random_state=42, shuffle=shuffle)
    return x_train, x_valid, x_test, y_train, y_valid, y_test

data = shuffle(data)
x_train, x_valid, x_test, y_train, y_valid, y_test = train_valid_test_split(data['tweets'], data['self-identity'])

train = pd.DataFrame({'label':y_train, 'x_train': x_train})
train.to_csv("/home/chuhan/ICWSM2023/tweets+retweets/dataforBert2_train.csv", index=False, sep='\t')
valid = pd.DataFrame({'label':y_valid, 'x_valid': x_valid})
valid.to_csv("/home/chuhan/ICWSM2023/tweets+retweets/dataforBert2_valid.csv", index=False, sep='\t')
test = pd.DataFrame({'label':y_test, 'x_test': x_test})
test.to_csv("/home/chuhan/ICWSM2023/tweets+retweets/dataforBert2_test.csv", index=False, sep='\t')


# training_dataframe = pd.DataFrame({'tweets':x_train,'self-identity':y_train})   
# training_dataframe.to_csv("/home/chuhan/ICWSM2023/tweets_only/dataforBert2_train.csv",index=False,sep=',')

# valid_dataframe = pd.DataFrame({'tweets':x_valid,'self-identity':y_valid})   
# valid_dataframe.to_csv("/home/chuhan/ICWSM2023/tweets_only/dataforBert2_valid.csv",index=False,sep=',')

# test_dataframe = pd.DataFrame({'tweets':x_test,'self-identity':y_test})   
# test_dataframe.to_csv("/home/chuhan/ICWSM2023/tweets_only/dataforBert2_test.csv",index=False,sep=',')


In [22]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle